# Data Science Team's Final Notebook

In [34]:
import pandas as pd
import ast

import spacy
import string
import re

import gensim

In [2]:
df_movies = pd.read_csv('../raw_data/movies_with_tokenized_plots.csv')

# Convert the string representation of lists back into actual lists of tokens
df_movies['wiki_plot_tokenized'] = df_movies['wiki_plot_tokenized'].apply(ast.literal_eval)

df_movies.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,wiki_plot_tokenized
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...","[bartender, work, saloon, serve, drink, custom..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","[moon, paint, smile, face, hang, park, night, ..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","[film, minute, long, compose, shot, girl, sit,..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"[second, consist, shot, shot, set, wood, winte..."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,"[early, know, adaptation, classic, fairytale, ..."


## American movies subset only

- it improves accuracy as measured by topic coherence of the LSI model (0.51 vs 0.38 for the set of all movies) and as seen when testing the similarity search function;

- the sizes of the models to be loaded for the search function  are smaller than for the whole set

- it helps the user looking for a movie title to first select the region of origin of the movie

In [3]:
am_movies = df_movies[df_movies['Origin/Ethnicity'] == 'American']
am_movies = am_movies.reset_index(drop=True)

In [4]:
#column names
am_movies.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'wiki_plot_tokenized'],
      dtype='object')

In [5]:
#Save the dataframe to use the titles and metadata for the search function
am_movies['Release Year'] = am_movies['Release Year'].astype('int16')  # Example for integers
am_titles = am_movies.drop(columns=['Plot', 'wiki_plot_tokenized'])
# Save the DataFrame as a Feather file
am_titles.to_feather('../models/am_titles.feather') # 1.4 MB

In [35]:
# Load the DataFrame from the Feather file
am_titles = pd.read_feather('../models/am_titles.feather')
am_titles.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page
0,1901,Kansas Saloon Smashers,American,Unknown,None,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...
1,1901,Love by the Light of the Moon,American,Unknown,None,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...
2,1901,The Martyred Presidents,American,Unknown,None,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,None,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_..."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",None,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...


In [7]:
print(f"Number of American movies: {len(am_movies)}")
print(f"Number of all movies: {len(df_movies)}")

Number of American movies: 17377
Number of all movies: 34886


In [8]:
am_movie_plot = am_movies['wiki_plot_tokenized']

In [9]:
from gensim import corpora

am_dictionary = corpora.Dictionary(am_movie_plot)

stoplist = set('hello and if this can would should could tell ask stop come go movie film'.split())
stop_ids = [am_dictionary.token2id[stopword] for stopword in stoplist if stopword in am_dictionary.token2id]
am_dictionary.filter_tokens(stop_ids)

In [10]:
# Save the dictionary to the ../models folder
am_dictionary.save('../models/am_dictionary.dict')

In [36]:
#Load the Gensim dictionary for the American movies subset
am_dictionary = corpora.Dictionary.load('../models/am_dictionary.dict')

In [12]:
import os

# File paths
am_dictionary_path = '../models/am_dictionary.dict'

# Calculate file sizes
am_dictionary_size = os.path.getsize(am_dictionary_path) / (1024 * 1024)      # Convert bytes to MB

# Print file sizes
print(f"Size of am_dictionary: {am_dictionary_size:.2f} MB")

Size of am_dictionary: 2.01 MB


In [37]:
from gensim import corpora

# Load the dictionary from the ../models folder
am_dictionary = corpora.Dictionary.load('../models/am_dictionary.dict')

In [14]:
am_corpus = [am_dictionary.doc2bow(desc) for desc in am_movie_plot]

In [15]:
import gensim
from gensim.models.coherencemodel import CoherenceModel

#Create a TF-IDF model

movie_tfidf_am_model = gensim.models.TfidfModel(am_corpus, id2word=am_dictionary)

In [16]:
# Save the TF-IDF model to the ../models folder
movie_tfidf_am_model.save('../models/movie_tfidf_am_model')

In [17]:
import os

# File paths
tfidf_am_model_path = '../models/movie_tfidf_am_model'

# Calculate file sizes
tfidf_am_model_size = os.path.getsize(tfidf_am_model_path) / (1024 * 1024)      # Convert bytes to MB

# # Print file sizes
# print(f"Size of movie_tfidf_am_model: {lsi_am_model_size:.2f} MB")

In [38]:
from gensim import models

# Load the TF-IDF model from the ../models folder
movie_tfidf_am_model = models.TfidfModel.load('../models/movie_tfidf_am_model')

In [19]:
print(movie_tfidf_am_model)

TfidfModel<num_docs=17377, num_nnz=2427674>


In [20]:
# Create the LSI model with 2000 topics (takes 2 min)
movie_lsi_am_model = gensim.models.LsiModel(movie_tfidf_am_model[am_corpus], id2word=am_dictionary, num_topics=2000)

When you save an `LsiModel` using the `save()` method, Gensim splits the model's data into multiple files for efficiency and modularity. The `projection.u.npy` file is one of these files, and it specifically stores the U matrix.

Gensim creates several files in the `../models` directory:

`movie_lsi_am_model` (main metadata file).
`movie_lsi_am_model.projection.u.npy` (left singular vectors).
`movie_lsi_am_model.projection.s.npy` (singular values).
`movie_lsi_am_model.state` (model state information).

When you load the `LsiModel` using the `load()` method Gensim automatically reads the `projection.u.npy` file (along with other associated files) to reconstruct the model. You typically don’t need to interact with this file directly unless you are debugging or analyzing the internal workings of the LSI model.



In [21]:
# Save the LSI model
movie_lsi_am_model.save('../models/movie_lsi_am_model')

In [22]:
import os

# File paths
lsi_am_model_path = '../models/movie_lsi_am_model'
lsi_am_model_projection_u_path = '../models/movie_lsi_am_model.projection.u.npy'
#lsi_am_model_projection_s_path = '../models/projection.s.npy'
#lsi_am_model_state_path = '../models/movie_lsi_am_model.state'


# Calculate file sizes
lsi_am_model_size = os.path.getsize(lsi_am_model_path) / (1024 * 1024)      # Convert bytes to MB
lsi_am_model_projection_u_size = os.path.getsize(lsi_am_model_projection_u_path) / (1024 * 1024)      # Convert bytes to MB
#lsi_am_model_projection_s_size = os.path.getsize(lsi_am_model_projection_s_path) / (1024 * 1024)      # Convert bytes to MB
#lsi_am_model_state_size = os.path.getsize(lsi_am_model_state_path) / (1024 * 1024)      # Convert bytes to MB

# Print file sizes
print(f"Size of movie_lsi_am_model: {lsi_am_model_size:.2f} MB")
print(f"Size of movie_lsi_am_model.projection.u: {lsi_am_model_projection_u_size:.2f} MB")
#print(f"Size of movie_lsi_am_model.projection.s: {lsi_am_model_projection_s_size:.2f} MB")
#print(f"Size of movie_lsi_am_model.state: {lsi_am_model_state_size:.2f} MB")

Size of movie_lsi_am_model: 2.70 MB
Size of movie_lsi_am_model.projection.u: 1302.49 MB


In [39]:
# Load the LSI model from the ../models folder
from gensim import models

movie_lsi_am_model = models.LsiModel.load('../models/movie_lsi_am_model')

In [24]:
# Transform the TF-IDF corpus into the LSI space
movie_lsi_am_corpus = movie_lsi_am_model[movie_tfidf_am_model[am_corpus]]

# Transform the original BoW corpus into the TF-IDF space
movie_tfidf_am_corpus = movie_tfidf_am_model[am_corpus]

In [25]:
# Serialize and Store the corpus locally
# (note the .index files created alongside the mm corpus files)
gensim.corpora.MmCorpus.serialize('../models/movie_tfidf_am_model_mm', movie_tfidf_am_model[am_corpus])
gensim.corpora.MmCorpus.serialize('../models/movie_lsi_am_model_mm',movie_lsi_am_model[movie_tfidf_am_model[am_corpus]])

In [26]:
import os

# File paths
tfidf_model_path = '../models/movie_tfidf_am_model_mm'
lsi_model_path = '../models/movie_lsi_am_model_mm'

# Calculate file sizes
tfidf_model_size = os.path.getsize(tfidf_model_path) / (1024 * 1024)  # Convert bytes to MB
lsi_model_size = os.path.getsize(lsi_model_path) / (1024 * 1024)      # Convert bytes to MB

# Print file sizes
print(f"Size of movie_tfidf_am_model_mm: {tfidf_model_size:.2f} MB")
print(f"Size of movie_lsi_am_model_mm: {lsi_model_size:.2f} MB")

Size of movie_tfidf_am_model_mm: 70.96 MB
Size of movie_lsi_am_model_mm: 1039.93 MB


In [40]:
#Load the indexed corpus
movie_tfidf_am_corpus = gensim.corpora.MmCorpus('../models/movie_tfidf_am_model_mm')
movie_lsi_am_corpus = gensim.corpora.MmCorpus('../models/movie_lsi_am_model_mm')

print(movie_tfidf_am_corpus)
print(movie_lsi_am_corpus)

MmCorpus(17377 documents, 85360 features, 2427674 non-zero entries)
MmCorpus(17377 documents, 2000 features, 34753997 non-zero entries)


In [41]:
#Load the MatrixSimilarity
from gensim.similarities import MatrixSimilarity

am_movie_index = MatrixSimilarity(movie_lsi_am_corpus, num_features = 2000)

In [42]:
# Define the tokenizer function for the user query

from spacy.lang.en.stop_words import STOP_WORDS

spacy_nlp = spacy.load('en_core_web_sm')

punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def tokenizer(sentence):
    sentence = re.sub('\'','',sentence)
    sentence = re.sub('\w*\d\w*','',sentence)
    sentence = re.sub(' +',' ',sentence)
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    sentence = re.sub(r'\n',' ',sentence)
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    tokens = spacy_nlp(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    return tokens

In [43]:
# Define the search function to get the most similar movies

from operator import itemgetter

def search_similar_movies(search_term):

    query_bow = am_dictionary.doc2bow(tokenizer(search_term))
    query_tfidf = movie_tfidf_am_model[query_bow]
    query_lsi = movie_lsi_am_model[query_tfidf]

    am_movie_index.num_best = 10

    movies_list = am_movie_index[query_lsi]

    movies_list.sort(key=itemgetter(1), reverse=True)
    movie_names = []

    for j, movie in enumerate(movies_list):

        movie_names.append (
            {
                'Relevance': round((movie[1] * 100),2),
                'Movie Title': am_titles['Title'][movie[0]],
                'Release Year': am_titles['Release Year'][movie[0]],
                'Director': am_titles['Director'][movie[0]],
                'Genre': am_titles['Genre'][movie[0]],
                'Wiki Page': am_titles['Wiki Page'][movie[0]]
            }

        )
        if j == (am_movie_index.num_best-1):
            break

    return pd.DataFrame(movie_names, columns=['Relevance','Movie Title', 'Release Year',
                                              'Director', 'Genre', 'Wiki Page'])

## Test searches

In [44]:
four_movies = am_movies[(am_movies['Title'] == 'Dark Shadows') |
            (am_movies['Title'] == 'Indiana Jones and the Kingdom of the Crystal Skull') |
            (am_movies['Title'] == 'True Crime') |
            (am_movies['Title'] == 'Jersey Boys')]
four_movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,wiki_plot_tokenized
13575,1999,True Crime,American,Clint Eastwood,"Clint Eastwood, Isaiah Washington, Denis Leary",crime drama,https://en.wikipedia.org/wiki/True_Crime_(1999...,"Steve Everett, an Oakland journalist recoverin...","[steve, everett, oakland, journalist, recover,..."
15461,2008,Indiana Jones and the Kingdom of the Crystal S...,American,Steven Spielberg,"Harrison Ford, Karen Allen, Shia LaBeouf, Cate...",adventure,https://en.wikipedia.org/wiki/Indiana_Jones_an...,"In 1957, Indiana Jones and his partner George ...","[indiana, jones, partner, george, mac, mchale,..."
16239,2012,Dark Shadows,American,Tim Burton,"Johnny Depp, Michelle Pfeiffer, Helena Bonham ...",horror comedy,https://en.wikipedia.org/wiki/Dark_Shadows_(film),"In 1760, the Collins family moves from Liverpo...","[collins, family, liverpool, england, maine, e..."
16811,2014,Jersey Boys,American,Clint Eastwood,John Lloyd Young\r\nErich Bergen\r\nVincent Pi...,musical,https://en.wikipedia.org/wiki/Jersey_Boys_(film),"In Belleville, New Jersey in 1951, Tommy DeVit...","[belleville, new, jersey, tommy, devito, perfo..."


### Synopses on the internet:

**Dark Shadows**
In 18th-century Maine, Barnabas Collins (Johnny Depp) presides over the town of Collinsport. A rich and powerful playboy, Barnabas seals his own doom when he breaks the heart of a witch named Angelique (Eva Green). Angelique turns Barnabas into a vampire and buries him alive. Two centuries later, Barnabas escapes from his tomb and finds 1972 Collinsport a very different place. His once-grand estate has fallen into ruin, and the dysfunctional remnants of his family have fared little better.

**Indiana Jones and the Kingdom of the Crystal Skull**
It's the height of the Cold War, and famous archaeologist Indiana Jones (Harrison Ford), returning from his latest adventure, finds out his job at Marshall College is in jeopardy. He meets Mutt (Shia La Beouf), a young man who wants Indy to help him find the legendary Crystal Skull of Akator, and the pair set out for Peru. However, deadly agent Irina Spalko (Cate Blanchett) is searching for the powerful artifact, too, because the Soviets believe it can help them conquer the world.

**True Crime**
Journalist Steve Everett (Clint Eastwood) has been sober two months, but things are still going badly. Steve's marriage to Barbara (Diane Venora) is barely holding together, and his editor (Denis Leary) hates him, since Steve has been having an affair with his wife. A chance assignment finds Everett interviewing death row inmate Frank Beachum (Isaiah Washington). When Everett uncovers evidence indicating Beachum may be innocent, he must race against time to uncover the truth.

**Jersey Boys**
In the 1960s, four scrappy young men from New Jersey -- Frankie Valli (John Lloyd Young), Bob Gaudio (Erich Bergen), Nick Massi (Michael Lomenda) and Tommy DeVito (Vincent Piazza) -- have the magic sound that propels them from singing under streetlights to singing in spotlights. With songs like "Sherry," "Big Girls Don't Cry" and "Walk Like a Man," the quartet finds itself at the top of the charts. However, personal and professional problems threaten to tear the group apart.

### Possible queries:

**Dark Shadows**
Johhny Depp as a vampire in the 18th century. A witch is in love with him.
horror comedy

**Indiana Jones and the Kingdom of the Crystal Skull**
Indiana Jones in the Cold War looking for a crystal skull.
adventure


**True Crime**
A journalist interviewing a death row inmate who is innocent.
A journalist, Eastwood, interviewing a death row inmate who is innocent.
A journalist has 12 hours to save a death row inmate who is innocent.
crime, drama

**Jersey Boys**
A musical about young men in a band that becomes famous.
musical

In [45]:
search_similar_movies("Indiana Jones in the Cold War looking for a skull.")

,Relevance,Movie Title,Release Year,Director,Genre,Wiki Page
0,66.59,Indiana Jones and the Kingdom of the Crystal S...,2008,Steven Spielberg,adventure,https://en.wikipedia.org/wiki/Indiana_Jones_an...
1,58.49,The Whole Town's Talking,1935,John Ford,comedy,https://en.wikipedia.org/wiki/The_Whole_Town%2...
2,49.71,Game of Death,2010,Giorgio Serafini,action,https://en.wikipedia.org/wiki/Game_of_Death_(2...
3,49.13,Bobby Jones: Stroke of Genius,2004,Rowdy Herrington,"biography, sports",https://en.wikipedia.org/wiki/Bobby_Jones:_Str...
4,48.98,Cinderella Jones,1946,Busby Berkeley,musical,https://en.wikipedia.org/wiki/Cinderella_Jones
5,47.78,Raiders of the Lost Ark,1981,Steven Spielberg,adventure,https://en.wikipedia.org/wiki/Raiders_of_the_L...
6,46.00,John Paul Jones,1959,John Farrow,biography,https://en.wikipedia.org/wiki/John_Paul_Jones_...
7,45.86,Come Live With Me,1941,Clarence Brown,comedy,https://en.wikipedia.org/wiki/Come_Live_with_M...
8,45.34,The Emperor Jones,1933,Dudley Murphy,drama,https://en.wikipedia.org/wiki/The_Emperor_Jone...
9,44.14,The Comedians,1967,Peter Glenville,drama,https://en.wikipedia.org/wiki/The_Comedians_(1...


In [33]:
search_similar_movies("Indiana Jones in the Cold War.")

,Relevance,Movie Title,Release Year,Director,Genre,Wiki Page
0,63.50,The Whole Town's Talking,1935,John Ford,comedy,https://en.wikipedia.org/wiki/The_Whole_Town%2...
1,59.73,Indiana Jones and the Kingdom of the Crystal S...,2008,Steven Spielberg,adventure,https://en.wikipedia.org/wiki/Indiana_Jones_an...
2,55.31,Game of Death,2010,Giorgio Serafini,action,https://en.wikipedia.org/wiki/Game_of_Death_(2...
3,54.80,Cinderella Jones,1946,Busby Berkeley,musical,https://en.wikipedia.org/wiki/Cinderella_Jones
4,54.28,Bobby Jones: Stroke of Genius,2004,Rowdy Herrington,"biography, sports",https://en.wikipedia.org/wiki/Bobby_Jones:_Str...
5,51.78,John Paul Jones,1959,John Farrow,biography,https://en.wikipedia.org/wiki/John_Paul_Jones_...
6,51.78,Come Live With Me,1941,Clarence Brown,comedy,https://en.wikipedia.org/wiki/Come_Live_with_M...
7,50.07,Raiders of the Lost Ark,1981,Steven Spielberg,adventure,https://en.wikipedia.org/wiki/Raiders_of_the_L...
8,49.29,The Emperor Jones,1933,Dudley Murphy,drama,https://en.wikipedia.org/wiki/The_Emperor_Jone...
9,48.23,The Comedians,1967,Peter Glenville,drama,https://en.wikipedia.org/wiki/The_Comedians_(1...


In [34]:
search_similar_movies('dinosaur adventure')

,Relevance,Movie Title,Release Year,Director,Genre,Wiki Page
0,49.50,Theodore Rex,1995,Jonathan Betuel,comedy,https://en.wikipedia.org/wiki/Theodore_Rex_(film)
1,45.09,We're Back! A Dinosaur's Story,1993,Simon Wells,animated,https://en.wikipedia.org/wiki/We%27re_Back!_A_...
2,42.80,The Lost World,1960,Irwin Allen,science-fiction,https://en.wikipedia.org/wiki/The_Lost_World_(...
3,39.35,Frenchman's Creek,1944,Mitchell Leisen,adventure,https://en.wikipedia.org/wiki/Frenchman%27s_Cr...
4,37.61,Dinosaurus!,1960,Irvin Yeaworth,sci-fi,https://en.wikipedia.org/wiki/Dinosaurus!
5,37.29,Jurassic Park,1993,Steven Spielberg,adventure,https://en.wikipedia.org/wiki/Jurassic_Park_(f...
6,35.47,Carnosaur 3: Primal Species,1996,Jonathan Winfrey,horror,https://en.wikipedia.org/wiki/Carnosaur_3:_Pri...
7,34.86,Future War,1997,Anthony Doublin,science fiction,https://en.wikipedia.org/wiki/Future_War
8,33.66,The Perils of Pauline,1914,Louis J. Gasnier,adventure serial,https://en.wikipedia.org/wiki/The_Perils_of_Pa...
9,32.68,Unknown Island,1948,Jack Bernhard,sci-fi,https://en.wikipedia.org/wiki/Unknown_Island


In [35]:
search_similar_movies("A journalist, Eastwood, interviewing a death row inmate who is innocent.")
#should be "True Crime" 1999

,Relevance,Movie Title,Release Year,Director,Genre,Wiki Page
0,42.27,Traci Townsend,2005,"Craig Ross, Jr.",comedy,https://en.wikipedia.org/wiki/Traci_Townsend
1,37.50,Riot in Cell Block 11,1954,Don Siegel,film noir,https://en.wikipedia.org/wiki/Riot_in_Cell_Blo...
2,37.09,The Last Mile,1959,Howard W. Koch,crime drama,https://en.wikipedia.org/wiki/The_Last_Mile_(1...
3,33.90,Prison,1987,Renny Harlin,horror,https://en.wikipedia.org/wiki/Prison_(1987_film)
4,33.90,Prison,1988,Renny Harlin,drama,https://en.wikipedia.org/wiki/Prison_(1988_film)
5,33.68,Pressure Point,1962,Hubert Cornfield,drama,https://en.wikipedia.org/wiki/Pressure_Point_(...
6,33.34,Inside the Walls of Folsom Prison,1951,Crane Wilbur,drama,https://en.wikipedia.org/wiki/Inside_the_Walls...
7,30.95,Hold Back Tomorrow,1955,Hugo Haas,drama,https://en.wikipedia.org/wiki/Hold_Back_Tomorrow
8,29.47,Behind the High Wall,1956,Abner Biberman,film noir,https://en.wikipedia.org/wiki/Behind_the_High_...
9,29.45,My Six Convicts,1952,Hugo Fregonese,comedy,https://en.wikipedia.org/wiki/My_Six_Convicts


In [36]:
search_similar_movies('Johhny Depp as a vampire in the 18th century. A witch is in love with him.')
#should be "Dark Shadows" 2014

,Relevance,Movie Title,Release Year,Director,Genre,Wiki Page
0,66.32,Brotherhood of Blood,2007,"Michael Roesch, Peter Scheerer",horror,https://en.wikipedia.org/wiki/Brotherhood_of_B...
1,61.00,Daughter of Darkness,1990,Stuart Gordon,horror,https://en.wikipedia.org/wiki/Daughter_of_Dark...
2,58.36,A Return to Salem's Lot,1987,Larry Cohen,horror,https://en.wikipedia.org/wiki/A_Return_to_Sale...
3,53.64,Blade,1998,Stephen Norrington,horror,https://en.wikipedia.org/wiki/Blade_(film)
4,52.14,Queen of the Damned,2002,Michael Rymer,horror,https://en.wikipedia.org/wiki/Queen_of_the_Dam...
5,50.60,Vampires,1998,John Carpenter,horror,https://en.wikipedia.org/wiki/Vampires_(film)
6,50.52,Burned at the Stake,1981,Bert I. Gordon,horror,https://en.wikipedia.org/wiki/Burned_at_the_Stake
7,49.60,30 Days of Night,2007,David Slade,horror,https://en.wikipedia.org/wiki/30_Days_of_Night...
8,49.00,Billy the Kid vs. Dracula,1966,William Beaudine,"western, horror",https://en.wikipedia.org/wiki/Billy_the_Kid_vs...
9,48.79,A Daughter of the Gods,1916,Herbert Brenon,drama,https://en.wikipedia.org/wiki/A_Daughter_of_th...
